# LIBS

In [99]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import os
import copy
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from operator import itemgetter
from collections import Counter

# IMPORT DATA FILES

In [85]:
#add data files location/workingdirectory here

def init_data(location=None):
    data=list()#################################################### list to store strings of txt file
    if location==None:
        location = os.getcwd()##################################### get current working directory location here    
    for file in os.listdir(location):
        try:
            if file.endswith(".txt"):
                with open(file,"r",encoding="utf-8") as f:
                    data.append(f.read())
        except Exception as e:
            raise "No files found"
    return data

# CLEANNING AND N-GRAM


In [86]:
def clean(data=None):
    if data==None or not data:
        print("No data to clean")
    else:
        try:################# can be shortened by regex ###########################
            final_token_stream=[]
            working_data=data.copy() ################################### WORKING DATA ######################################
            stoplist = set(stopwords.words('english') + list(punctuation) + list(['“','”']))
            nlp = spacy.load("en_core_web_sm")
            clean_data=[]
            for i in working_data:
                i=i.replace("\ufeff","").replace("\t"," ").replace("_","").replace(".","").replace(",","").strip()
                #i=i.replace("'ll"," will").replace("'s"," is").replace("'re"," are").replace("'m"," am")
                i=re.sub("[\n]+\s?"," ",i)
                i=i.lower();
                clean_data.append(i)
            for i in clean_data:
                out=nlp(i)
                #for token in out:
                 #   print(token, token.pos_, token.lemma_, token.text);
    # BUILD PROPERNOUN AND ADVERBS LIST #
                proper_noun=set([token.text for token in out if token.pos_ == "PROPN"])
                adverb=set([token.text for token in out if token.pos_ == "ADV"])
    # BUILD ELIMINATION LIST OF WORDS #
                eliminate=[]
                eliminate.extend(stoplist)
                eliminate.extend(list(proper_noun))
                eliminate.extend(list(adverb))
                #print(eliminate);
    # TOKENS AFTER ELIMINATION #
                new_token_stream=""
                for token in out:
                    if (token.pos_!="PUNCT" and token.pos_!="SPACE" and token.lemma_!="-PRON-"):
                        new_token_stream+=token.lemma_+" "
                    elif token.lemma_=="-PRON-":
                        new_token_stream+=token.text+" "
                        
                tokens = [token for token in nltk.word_tokenize(new_token_stream) if token not in eliminate]
                final_token_stream.append(tokens)
                #print(new_token_stream);
            return final_token_stream
        except Exception as e:
            raise e

# Build HASH table

In [135]:
def build_hash_table(data=None):
    if data==None or (not data):
        print("No data to build table")
    else:
        try:
            grams_hash = {}
            for i in data:
                three_grams_list.extend(list(ngrams(i, 3)))
            grams_hash=Counter(three_grams_list)
           # sorted_grams_hash=sorted(grams_hash.items(), key=itemgetter(1),reverse=True)
            return grams_hash
        except Exception as e:
            raise e
        '''try:
            grams_hash = {}
            for i in data:
                three_grams_list=list(ngrams(i, 3))
                for gram in three_grams_list:
                    if gram not in grams_hash.keys():
                        grams_hash.update({gram:1})
                    else:
                        gram_occurrences = grams_hash[gram]
                        grams_hash.update({gram:gram_occurrences+1})
            sorted_grams_hash=sorted(grams_hash.items(), key=itemgetter(1),reverse=True)
            return sorted_grams_hash'''
        

In [140]:
#unstable takes long time, needs more thought, wrongly coded
def cosine_similarity(a, b):
    for i in cleaned_test:
        a.extend(list(ngrams(i, 3)))
    vec1 = Counter(a)
    vec2 = b
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    return float(numerator) / denominator

# FOR TESTING ONLY

# METHODS DEFINED
#### -> init\_data(foo)  : "foo" type:string, location of working directory; returns list of strings
#### -> clean(foo)  : "foo" type:list of obtained data strings from init\_data; returns list of cleaned  strings
#### -> build\_hash\_table(foo) : "foo" type:list of obtained tokenised strings from clean(); returns dictionary having key:value\::token:count desc

$$TEST$$

In [88]:
data=init_data()

In [89]:
data;

In [90]:
cleaned_data=clean(data)
len(cleaned_data)

4

In [136]:
hash_table=build_hash_table(cleaned_data)

In [137]:
len(hash_table)

100320

In [138]:
type(hash_table)

collections.Counter

In [94]:
test_string=["He had concluded that pigs must be able to fly in Hog Heaven.","I would be delighted if the sea were full of cucumber juice.",\
             "If I don’t like something, I’ll stay away from it.","At that moment she realized she had a sixth sense."\
             "Italy is my favorite country; in fact, I plan to spend two weeks there next year."]

In [95]:
test_string

['He had concluded that pigs must be able to fly in Hog Heaven.',
 'I would be delighted if the sea were full of cucumber juice.',
 'If I don’t like something, I’ll stay away from it.',
 'At that moment she realized she had a sixth sense.Italy is my favorite country; in fact, I plan to spend two weeks there next year.']

In [96]:
cleaned_test=clean(test_string)

In [103]:
cleaned_test

[['conclude', 'pig', 'must', 'able', 'fly'],
 ['would', 'delight', 'sea', 'full', 'cucumber', 'juice'],
 ['like', 'something', 'stay'],
 ['moment',
  'realize',
  'sixth',
  'senseitaly',
  'favorite',
  'country',
  'fact',
  'plan',
  'spend',
  'two',
  'week',
  'next',
  'year']]

In [142]:
cs=cosine_similarity(cleaned_test,hash_table)


KeyboardInterrupt



In [124]:
a=[]
for i in cleaned_test:
    a.extend(list(ngrams(i, 3)))
vec1 = Counter(a)
vec1

Counter({('conclude', 'pig', 'must'): 1,
         ('pig', 'must', 'able'): 1,
         ('must', 'able', 'fly'): 1,
         ('would', 'delight', 'sea'): 1,
         ('delight', 'sea', 'full'): 1,
         ('sea', 'full', 'cucumber'): 1,
         ('full', 'cucumber', 'juice'): 1,
         ('like', 'something', 'stay'): 1,
         ('moment', 'realize', 'sixth'): 1,
         ('realize', 'sixth', 'senseitaly'): 1,
         ('sixth', 'senseitaly', 'favorite'): 1,
         ('senseitaly', 'favorite', 'country'): 1,
         ('favorite', 'country', 'fact'): 1,
         ('country', 'fact', 'plan'): 1,
         ('fact', 'plan', 'spend'): 1,
         ('plan', 'spend', 'two'): 1,
         ('spend', 'two', 'week'): 1,
         ('two', 'week', 'next'): 1,
         ('week', 'next', 'year'): 1})